### Data ingestion

In [2]:
from langchain_core.documents import Document

In [3]:
doc  = Document(
  page_content="This is the main content Im using to create RAG",
  metadata={
    "source":"nothing.txt",
    "Auther":"Akash VP",
    "pages":1,
    "date_created":"11-07-2025"
  }
  
)

In [4]:
#text loader
from langchain_community.document_loaders import TextLoader

loader = TextLoader("../data/simple/policy.txt",encoding = "utf-8")
document = loader.load()

print(document)


/home/bititude/LANG_CHAIN/RAG_practice_1/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Document(metadata={'source': '../data/simple/policy.txt'}, page_content='# Company Policies Document\n\n## 1. Work-Life Balance\nAt TechCorp, we strongly encourage employees to maintain a healthy work-life balance.\nEmployees can work flexible hours between 8 AM to 8 PM, as long as they complete their assigned 8-hour workday.\nWork-from-home is allowed on Fridays and up to 4 additional days per month upon manager approval.\n\n## 2. Leave Policy\nEach full-time employee is entitled to 20 days of paid leave per year.\nUnused leaves can be carried forward up to a maximum of 10 days.\nFor emergencies, employees can request up to 5 additional unpaid leaves.\nPublic holidays are separate and do not count toward paid leave.\n\n## 3. Remote Work Policy\nRemote work is permitted for roles that do not require physical presence in the office.\nEmployees must ensure a reliable internet connection and a professional workspace.\nMeetings and performance reviews will continue virtually during remote

In [5]:
#Directory loader
from langchain_community.document_loaders import DirectoryLoader

#for lading all the text file from the directory
dir_loader = DirectoryLoader(
  "../data/simple",
  glob="**/*.txt",
  loader_cls=TextLoader,
  loader_kwargs={"encoding":"utf-8"},
  show_progress=True
)

documents = dir_loader.load()
print(documents)

100%|██████████| 2/2 [00:00<00:00, 2512.31it/s]

[Document(metadata={'source': '../data/simple/policy.txt'}, page_content='# Company Policies Document\n\n## 1. Work-Life Balance\nAt TechCorp, we strongly encourage employees to maintain a healthy work-life balance.\nEmployees can work flexible hours between 8 AM to 8 PM, as long as they complete their assigned 8-hour workday.\nWork-from-home is allowed on Fridays and up to 4 additional days per month upon manager approval.\n\n## 2. Leave Policy\nEach full-time employee is entitled to 20 days of paid leave per year.\nUnused leaves can be carried forward up to a maximum of 10 days.\nFor emergencies, employees can request up to 5 additional unpaid leaves.\nPublic holidays are separate and do not count toward paid leave.\n\n## 3. Remote Work Policy\nRemote work is permitted for roles that do not require physical presence in the office.\nEmployees must ensure a reliable internet connection and a professional workspace.\nMeetings and performance reviews will continue virtually during remote

In [6]:
# loading pdf
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

dir_loader = DirectoryLoader(
  "../data/pdf",
  glob="**/*.pdf",
  loader_cls=PyMuPDFLoader,
  show_progress=False
)
pdf_doc = dir_loader.load()
print(pdf_doc)

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-04-03T10:53:24+05:30', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'total_pages': 10, 'format': 'PDF 1.7', 'title': '', 'author': 'Apoorva Satheesh R K', 'subject': '', 'keywords': '', 'moddate': '2025-04-03T10:53:24+05:30', 'trapped': '', 'modDate': "D:20250403105324+05'30'", 'creationDate': "D:20250403105324+05'30'", 'page': 0}, page_content='BITITUDE TECHNOLOGIES \n \n \nEMPLOYEE HANDBOOK \n2025'), Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-04-03T10:53:24+05:30', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'total_pages': 10, 'format': 'PDF 1.7', 'title': '', 'author': 'Apoorva Satheesh R K', 'subject': '', 'keywor

### RAG_pipeline: data ingestion


In [7]:
from langchain_core.documents import Document
from langchain_community.document_loaders import TextLoader
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import os
from pathlib import Path


from dotenv import load_dotenv

load_dotenv()

True

In [8]:
def process_all_pdf(pdf_dir):
  all_docs = []
  pdf_dir = Path(pdf_dir)

  dir_loader = DirectoryLoader(
    pdf_dir,
    glob="**/*.pdf",
    loader_cls=PyMuPDFLoader,
    show_progress=False
  )
  all_documents = dir_loader.load()
  print(f"found {len(all_documents)}")

  return all_documents


docs = process_all_pdf("../data/pdf")
  

found 14


In [9]:
# # loading pdf
# from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader

# dir_loader = PyMuPDFLoader(
#    )
# pdf_doc = dir_loader.load()
# print(pdf_doc)

In [10]:
## text splitting

def split_documents(documents,chunk_size = 1000, chunk_overlap = 200):
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=chunk_size,
    chunk_overlap=chunk_overlap,
    length_function = len,
    separators=["\n\n","\n"," ",""]
  )

  split_docs = text_splitter.split_documents(documents)
  print(f"split {len(documents)} to {len(split_docs)} chunks ")

  # if split_docs:
  #   print(split_docs)
  return split_docs

In [11]:
doc_chunks = split_documents(docs)

split 14 to 32 chunks 


### Embedding and vector store

In [12]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List,Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [13]:
class EmbeddingMnager:

    def __init__(self, model_name: str = "sentence-transformers/all-MiniLM-L6-v2"):
        """
        Ineatialize the embedding manager

        Args:
        model name: Hugging fce model name for sentence embedding

        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        try:
            self.model = SentenceTransformer(self.model_name,token= os.getenv('HF_KEY'))
            print(f"model loaded succesfully{self.model}")

        except Exception as e:
            print("Loadding errorrrr",e)

    def generate_embedding(self, texts:List[str])-> np.ndarray:
        if not self.model:
            print("model errorr")

        print("generating embedding...")
        embeddings  = self.model.encode(texts, show_progress_bar=True)
        return embeddings

embedding_manager = EmbeddingMnager()
embedding_manager

model loaded succesfullySentenceTransformer(
  (0): Transformer({'max_seq_length': 256, 'do_lower_case': False, 'architecture': 'BertModel'})
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)


### Vector store

In [45]:
class VectorStore:

    def __init__(
      self,
      collection_name: str = "pdf_documents",
      persist_directory: str = "../data/vector_sore",):

        """ Ineatialize vector store
        Args:
            Collection_name: Name of the collection
            persist_directory: location of vector DB
        """

        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Ineatialize chromadb client and collection"""

        try:
            os.makedirs(self.persist_directory,exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)

            self.collection = self.client.get_or_create_collection(
        name = self.collection_name,
        metadata={"description":"PFD document embeddings for RAG"}
      )

        except Exception as e:
            print("Error found !! cant create collection",e)

    def add_documents(self, documents:List[Any], embeddings: np.ndarray):
        """ Add document and their embedding to vector store
        
        Args:
            documents: List of langchain documents
            embeddings: embeddings for the document
        """

        if len(documents) != len(embeddings):
            raise ValueError("Number of document must match number of embedding")

        # adding data
        ids = []
        metadatas = []
        document_text = []
        embedding_list = []

        for i, (doc,embedding) in enumerate(zip(documents,embeddings)):
            
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)

            # prepare metadata
            metadata  = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)

            document_text.append(doc.page_content)
            print(embedding,type(embedding))
            embedding_list.append(embedding.tolist())

        try:
            self.collection.add(
                ids = ids,
                embeddings=embedding_list,
                metadatas=metadatas,
                documents = document_text
            )
            print("data added succesfullyyyy")

        except Exception as e:
            print("error founf while adding data!!!",e)


vector_store = VectorStore()
vector_store

In [15]:
doc_chunks

[Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-04-03T10:53:24+05:30', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'total_pages': 10, 'format': 'PDF 1.7', 'title': '', 'author': 'Apoorva Satheesh R K', 'subject': '', 'keywords': '', 'moddate': '2025-04-03T10:53:24+05:30', 'trapped': '', 'modDate': "D:20250403105324+05'30'", 'creationDate': "D:20250403105324+05'30'", 'page': 0}, page_content='BITITUDE TECHNOLOGIES \n \n \nEMPLOYEE HANDBOOK \n2025'),
 Document(metadata={'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'creationdate': '2025-04-03T10:53:24+05:30', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'total_pages': 10, 'format': 'PDF 1.7', 'title': '', 'author': 'Apoorva Satheesh R K', 'subject': '', 'keywo

In [46]:
#convert text to embedding
texts = [doc.page_content for doc in doc_chunks]

#Generate the embedding
embeddings = embedding_manager.generate_embedding(texts)

# #store vectordb
vector_store.add_documents(doc_chunks,embeddings)

generating embedding...


Batches: 100%|██████████| 1/1 [00:01<00:00,  1.01s/it]

[-7.39254504e-02  5.88736683e-02 -2.49391254e-02 -2.84897443e-02
 -8.54014680e-02 -2.58661471e-02  7.39482865e-02 -6.65354449e-03
 -1.73461363e-01 -1.35493921e-02 -1.66859804e-03 -1.69258323e-02
  3.65641192e-02 -1.89000685e-02 -1.58022232e-02  3.63824591e-02
  4.96816002e-02 -7.10407123e-02  2.57761311e-02 -1.37973502e-01
 -9.39077139e-03 -2.08015442e-02  2.97965948e-02 -2.80137900e-02
 -1.91581883e-02  3.26401700e-04  9.65943187e-03 -2.42045224e-02
  2.41768230e-02 -5.31884581e-02 -2.28842180e-02  9.97547582e-02
  7.24783614e-02 -2.83117723e-02  1.05507663e-02  4.30480428e-02
  7.91528746e-02  4.98839393e-02 -7.78129790e-03  1.66786462e-02
 -8.52254033e-02 -1.20037742e-01 -1.41728688e-02 -1.83501486e-02
  1.99597576e-04 -6.69934042e-03  2.58138347e-02  3.85873904e-03
 -5.38289659e-02  4.18464132e-02  2.25225985e-02 -4.06521298e-02
  7.99246281e-02  5.08814305e-02 -3.89810391e-02  6.39339583e-03
  3.88184786e-02  1.58659685e-02 -7.73824453e-02  3.73708121e-02
 -3.77396378e-03 -3.87617

### RAG- Retriever pipeline from VectorStore

In [17]:
class RAGRetriever:

    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingMnager):

        """ Ineatialize vector store
    
    Args:
      vector_store: vector db containing document embedding
      embedding_manager: Manager for generating embedding

    """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager 

    def retrieve(self,query:str, top_k:int = 5, score_threshold:float = 0) -> List[Dict[str,Any]]:
        """ Retriebe relecvent document from query
    
    Args:
      query: The search query
      top_k: Number of top result to return
      score_threshold: Minimum similarity score threshold

    Returns:
      List of dictionaries containing retrieved documents and metadata
    """

        query_embedding = self.embedding_manager.generate_embedding([query])[0]

        try:
            results = self.vector_store.collection.query(
        query_embeddings=[query_embedding.tolist()],
        n_results= top_k
      )
            retrieved_docs = []

            print(results["metadatas"][0])

            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results["metadatas"][0]
                distances = results['distances'][0]
                ids = results['ids'][0]

                for i,(doc_id,document, metadata,distance) in enumerate(zip(ids,documents,metadatas,distances)):
                    similarity_score = 1-distance

                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                          "id":doc_id,
                          "content":document,
                          "metadata":metadata,
                          "similarity_score":similarity_score,
                          "rank":i+1
                     })
            else:
                print("No documnet found!")

        except Exception as e:
            print("Errorr while retrieving..",e)
            return []
        return retrieved_docs


rag_retriever = RAGRetriever(vector_store, embedding_manager)

In [18]:
rag_retriever.retrieve("what is attempt to murder")

generating embedding...


Batches: 100%|██████████| 1/1 [00:00<00:00, 68.67it/s]

[{'producer': 'Online2PDF.com', 'moddate': '', 'source': '../data/pdf/repealedfileopen.pdf', 'trapped': '', 'total_pages': 119, 'creationdate': '2023-06-28T10:58:56+02:00', 'title': '', 'file_path': '../data/pdf/repealedfileopen.pdf', 'author': '', 'subject': '', 'format': 'PDF 1.4', 'doc_index': 407, 'modDate': '', 'content_length': 959, 'creator': 'Online2PDF.com', 'page': 73, 'keywords': '', 'creationDate': "D:20230628105856+02'00'"}, {'source': '../data/pdf/repealedfileopen.pdf', 'file_path': '../data/pdf/repealedfileopen.pdf', 'producer': 'Online2PDF.com', 'keywords': '', 'creationDate': "D:20230628105856+02'00'", 'creationdate': '2023-06-28T10:58:56+02:00', 'subject': '', 'trapped': '', 'moddate': '', 'page': 73, 'format': 'PDF 1.4', 'doc_index': 407, 'author': '', 'creator': 'Online2PDF.com', 'modDate': '', 'title': '', 'total_pages': 119, 'content_length': 959}, {'creationDate': "D:20230628105856+02'00'", 'total_pages': 119, 'keywords': '', 'modDate': '', 'content_length': 959,

[{'id': 'doc_3bc7e985_407',
  'content': "(f) Z strikes B. B is by this provocation excited to violent rage. A, a bystander, intending to take advantage of B's rage, and \nto cause him to kill Z, puts a knife into B's hand for that purpose. B kills Z with the knife. Here B may have committed only \nculpable homicide, but A is guilty of murder. \nException 2.—Culpable homicide is not murder if the offender, in the exercise in good faith of the \nright of private defence of person or property, exceeds the power given to him by law and causes the \ndeath of the person against whom he is exercising such right of defence without premeditation, and \nwithout any intention of doing more harm than is necessary for the purpose of such defence. \nIllustration \nZ attempts to horsewhip A, not in such a manner as to cause grievous hurt to A. A draws out a pistol. Z persists in the \nassault. A believing in good faith that he can by no other means prevent himself from being horsewhipped, shoots Z d

### Vectordb context pipeline with llm output

In [19]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
from dotenv import load_dotenv
load_dotenv()

#using gooole gemini llm
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash", api_key=os.getenv("GEMINI_API_KEY")
)

def rag_simple(query:str, retriever:RAGRetriever, llm, top_k =3):
  """ This will retriev context and give it to llm for generate output
  
  Args:
    query: user query
    retriever: Retriver for getting context
    llm: LLM model
    top_k: number of ouptut from vectordb
    
  Returns:
    LLM response content  
  """
  
  results = retriever.retrieve(query, top_k = top_k)
  context: str = "\n\n".join([doc['content'] for doc in results]) if results else ""

  if not context:
    return "No relevent context found!"
  
  #generate answer using llm
  prompt = f"""Answer for the question using the following context
            context:{context}
            question:{query}
            Answer:
            """
  response = llm.invoke([prompt.format(context = context, query = query)])

  return response.content
  



In [20]:
rag_simple("what is the work hour", rag_retriever, llm)

generating embedding...


Batches: 100%|██████████| 1/1 [00:00<00:00, 93.64it/s]

[{'creationdate': '2025-04-03T10:53:24+05:30', 'modDate': "D:20250403105324+05'30'", 'doc_index': 12, 'page': 5, 'moddate': '2025-04-03T10:53:24+05:30', 'author': 'Apoorva Satheesh R K', 'keywords': '', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'total_pages': 10, 'creator': 'Microsoft® Word 2021', 'format': 'PDF 1.7', 'trapped': '', 'producer': 'Microsoft® Word 2021', 'subject': '', 'content_length': 975, 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'creationDate': "D:20250403105324+05'30'", 'title': ''}, {'title': '', 'moddate': '2025-04-03T10:53:24+05:30', 'subject': '', 'creationdate': '2025-04-03T10:53:24+05:30', 'modDate': "D:20250403105324+05'30'", 'format': 'PDF 1.7', 'total_pages': 10, 'creator': 'Microsoft® Word 2021', 'content_length': 975, 'author': 'Apoorva Satheesh R K', 'page': 5, 'producer': 'Microsoft® Word 2021', 'creationDate': "D:20250403105324+05'30'", 'source': '../data/pdf/Bititude Technologies Employee Handboo

'The standard working hours are Monday to Friday, from 09:00 to 18:00.'

### Enhanced RAG pipeline

In [21]:
def rag_advanced(query,retriever,llm,top_k = 5, min_score = 0.0, return_context = False):
    """RAG with extra feature
  
  Args:
    query: user query
    retriever: Retriver for getting context
    llm: LLM model
    top_k: number of ouptut from vectordb
    
  Returns:
    LLM response content  
  
  """

    results = retriever.retrieve(query, top_k=5, score_threshold=min_score)
    print(results)
    if not results:
        return {
            "answer": "No relevent context found.",
            "source": [],
            "confidence": 0.0,
            "context": "",
        }

    # prepare context and source
    context = "\n\n".join([doc["content"] for doc in results]) if results else ""
    sources = [
        {
            "source": doc["metadata"].get("source", "Unknown"),
            "page": doc["metadata"].get("Page", "Unknown"),
            "score": doc["similarity_score"],
            "preview": doc["content"][:120]+"..."

        }for doc in results] #applying throughout the results
    confidence = max([doc['similarity_score'] for doc in results])
    
    #Generating output
    prompt = f"""Use the following context to answer for the given question consisely
     
    context:{context}

    question:{query}

    Answer:
     """
    
    response = llm.invoke([prompt.format(context = context, query = query)])

    output = {
        "answer":response.content,
        "sources":sources,
        "confidence": confidence
    }
    
    if return_context:
        output['context'] = context
    
    return output
    

In [22]:
result = rag_advanced("what is the work hour", rag_retriever, llm,  top_k = 3, min_score=0.0, return_context=True)

result

generating embedding...


Batches: 100%|██████████| 1/1 [00:00<00:00, 28.46it/s]

[{'title': '', 'trapped': '', 'content_length': 975, 'page': 5, 'format': 'PDF 1.7', 'modDate': "D:20250403105324+05'30'", 'total_pages': 10, 'moddate': '2025-04-03T10:53:24+05:30', 'creator': 'Microsoft® Word 2021', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'author': 'Apoorva Satheesh R K', 'doc_index': 12, 'keywords': '', 'producer': 'Microsoft® Word 2021', 'creationDate': "D:20250403105324+05'30'", 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'subject': '', 'creationdate': '2025-04-03T10:53:24+05:30'}, {'modDate': "D:20250403105324+05'30'", 'producer': 'Microsoft® Word 2021', 'keywords': '', 'creationdate': '2025-04-03T10:53:24+05:30', 'subject': '', 'creator': 'Microsoft® Word 2021', 'format': 'PDF 1.7', 'content_length': 975, 'author': 'Apoorva Satheesh R K', 'moddate': '2025-04-03T10:53:24+05:30', 'trapped': '', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'page': 5, 'creationDate': "D:20250403105324+05

{'answer': 'Standard working hours are Monday to Friday, from 09:00 to 18:00.',
 'sources': [{'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf',
   'page': 'Unknown',
   'score': 0.06925702095031738,
   'preview': '4 | P a g e  \n \n \n5.2 Standard Working Hours and Break Times \n \nStandard Working Hours: Our standard working hours are a...'},
  {'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf',
   'page': 'Unknown',
   'score': 0.06925702095031738,
   'preview': '4 | P a g e  \n \n \n5.2 Standard Working Hours and Break Times \n \nStandard Working Hours: Our standard working hours are a...'},
  {'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf',
   'page': 'Unknown',
   'score': 0.06925702095031738,
   'preview': '4 | P a g e  \n \n \n5.2 Standard Working Hours and Break Times \n \nStandard Working Hours: Our standard working hours are a...'},
  {'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf',
   'page': 'Unk

### Advanced RAG pipeline

In [43]:
import time


class AdvancedRAGPipeline:
    def __init__(self,retriever,llm):
        """ setting retriever and llm

        Args:
            retriever:
            llm:

        """
        self.retriever = retriever
        self.llm = llm
        self.history = [] #for storing query history

    def query(self,question:str, top_k = 5, min_score: float = 0.1, stream:bool = False, summarize:bool = False):
        """This will return the response for a query
    
          Args:
            quetion:
            top_k:
            min_score:
            stream:
        """
        results = self.retriever.retrieve(
            question, top_k=top_k, score_threshold=min_score
        )

        if not results:
            answer = "No relevent context found!"
            sources = []
            context = ""
        else:
            context = "\n\n".join(doc["content"] for doc in results)
            sources = [
                {
                    "source": doc["metadata"].get("source", "Unknown"),
                    "page": doc["metadata"].get("Page", "Unknown"),
                    "score": doc["similarity_score"],
                    "preview": doc["content"][:120] + "...",
                }
                for doc in results
            ]  

            prompt = f"""Use the following context to answer the question consisely
            \n context:{context}
            \n question:{question}
            """
            if stream:
                print("streaming..")
                for i in range(0,len(prompt),50):
                    print(prompt[i:i+50],end = "",flush=True)
                    time.sleep(0.05)
                print()

            response = self.llm.invoke([prompt.format(context = context, question = question)])
            answer = response.content



            citations = [f"[{i+1}] {src['source']} (page {src['page']})" for i,src in enumerate(sources)]
            answer_with_citations = answer+"\n\nCitations:\n:"+"\n".join(citations) if citations else answer

            # summarize
            summary = None
            if summarize and answer:
                summary_prompt = f"Summarize the following answer in 2 sentences:\n{answer}"
                summary_resp = self.llm.invoke([summary_prompt])
                summary = summary_resp.content

            # store query history
            self.history.append({
                "question":question,
                "answer":answer,
                "sources":sources,
                "summary":summary
            })

            return {
                "question": question,
                "answer": answer_with_citations,
                "sources": sources,
                "summary": summary,
                "history":self.history
            }


avd_rag = AdvancedRAGPipeline(rag_retriever,llm)
result = avd_rag.query("What is the leave policy",stream=True)
print(result)

generating embedding...


Batches: 100%|██████████| 1/1 [00:00<00:00, 94.01it/s]

[{'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'page': 4, 'author': 'Apoorva Satheesh R K', 'creationDate': "D:20250403105324+05'30'", 'content_length': 212, 'modDate': "D:20250403105324+05'30'", 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'doc_index': 11, 'subject': '', 'moddate': '2025-04-03T10:53:24+05:30', 'keywords': '', 'creationdate': '2025-04-03T10:53:24+05:30', 'title': '', 'trapped': '', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'format': 'PDF 1.7', 'total_pages': 10}, {'author': 'Apoorva Satheesh R K', 'total_pages': 10, 'subject': '', 'file_path': '../data/pdf/Bititude Technologies Employee Handbook.pdf', 'doc_index': 11, 'trapped': '', 'modDate': "D:20250403105324+05'30'", 'title': '', 'page': 4, 'producer': 'Microsoft® Word 2021', 'creator': 'Microsoft® Word 2021', 'format': 'PDF 1.7', 'moddate': '2025-04-03T10:53:24+05:30', 'source': '../data/pdf/Bititude Technologies Employee Handbook.pdf',

nal time off, should be formally 
requested in advance through the approved company channels.

facilitate coverage and preserve operational efficiency. 
 
Leave Requests: Planned absences, such as vacation or personal time off, should be formally 
requested in advance through the approved company channels.

facilitate coverage and preserve operational efficiency. 
 
Leave Requests: Planned absences, such as vacation or personal time off, should be formally 
requested in advance through the approved company channels.

facilitate coverage and preserve operational efficiency. 
 
Leave Requests: Planned absences, such as vacation or personal time off, should be formally 
requested in advance through the approved company channels.

facilitate coverage and preserve operational efficiency. 
 
Leave Requests: Planned absences, such as vacation or personal time off, should be formally 
requested in advance through the approved company channels.
            
 question:What is the leave policy
  